<a href="https://colab.research.google.com/github/TheAzouz/Project_ADA/blob/main/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd "drive/My Drive/Colab Notebooks/ADA_rami"

/content/drive/My Drive/Colab Notebooks/ADA_rami


In [4]:
import csv
import pandas as pd
import numpy as np



In [5]:
ls

training_test_set_anonymized.csv


In [6]:
df=pd.read_csv("training_test_set_anonymized.csv",sep=',')
df.head()

,Unnamed: 0,checkin_ratio,daily_total_checkin_rate,end_of_day_ratio,end_of_inactive_day_ratio,is_home,kilometer_distance_to_most_checked_in,midnight_ratio,num_of_total_checkins,number_of_checkins_at_this_location,page_rank,reverse_page_rank,user_id
0,0,0.086957,10.003725,0.000000,0.000000,False,6.833662,0.0,23,2,0.023198,0.114258,1
1,1,0.043478,10.003725,0.000000,0.000000,False,8.456719,0.0,23,1,0.033057,0.061761,1
2,2,0.434783,10.003725,0.666667,0.666667,True,0.000000,1.0,23,10,0.294426,0.245317,1
3,3,0.086957,10.003725,0.000000,0.000000,False,0.326270,0.0,23,2,0.123303,0.111886,1
4,4,0.130435,10.003725,0.000000,0.000000,False,0.707318,0.0,23,3,0.175707,0.176912,1


In [7]:
trial=df.groupby('user_id').agg({'daily_total_checkin_rate':'std'})
trial.head()

,daily_total_checkin_rate
user_id,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


In [8]:
df_tmp=df[['checkin_ratio','end_of_day_ratio','end_of_inactive_day_ratio','midnight_ratio','kilometer_distance_to_most_checked_in','page_rank','reverse_page_rank','is_home']]
df_tmp.head()

,checkin_ratio,end_of_day_ratio,end_of_inactive_day_ratio,midnight_ratio,kilometer_distance_to_most_checked_in,page_rank,reverse_page_rank,is_home
0,0.086957,0.000000,0.000000,0.0,6.833662,0.023198,0.114258,False
1,0.043478,0.000000,0.000000,0.0,8.456719,0.033057,0.061761,False
2,0.434783,0.666667,0.666667,1.0,0.000000,0.294426,0.245317,True
3,0.086957,0.000000,0.000000,0.0,0.326270,0.123303,0.111886,False
4,0.130435,0.000000,0.000000,0.0,0.707318,0.175707,0.176912,False


In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [171]:
X=df[['checkin_ratio','end_of_day_ratio','end_of_inactive_day_ratio','midnight_ratio',\
      'kilometer_distance_to_most_checked_in','page_rank','reverse_page_rank']].values
Y=df_tmp['is_home'].values

X_train, X_test, y_train, y_test = train_test_split(preprocessing.scale(X), Y, test_size=0.2, random_state=0)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score,precision_score,recall_score

In [187]:
clf = make_pipeline(StandardScaler(), SVC(kernel='poly',degree=3))
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [188]:
y_pred=clf.predict(X_test)

In [189]:
f1_score(y_test,y_pred)

0.7904761904761904

In [190]:
precision_score(y_test,y_pred)

0.8736842105263158

In [191]:
recall_score(y_test,y_pred)

0.7217391304347827

In [141]:
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Input,LeakyReLU,ReLU
from keras.optimizers import SGD
from keras.optimizers import *
from keras import backend as K

In [134]:
from keras import backend as K

#compute recall
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

#compute precision
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

#compute F1-score
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [182]:
input=Input((X_train.shape[1]))
d1=Dense(4)(input)
d1=LeakyReLU(0.3)(d1)
d1=Dropout(0.5)(d1)
d2=Dense(16)(d1)
d2=LeakyReLU(0.3)(d2)
d2=Dropout(0.5)(d2)
d3=Dense(64)(d2)
d3=ReLU(0.3)(d3)
d3=Dropout(0.5)(d3)
d4=Dense(2, activation='sigmoid')(d3)
d4=Dropout(0.3)(d4)
model=Model(inputs=input,outputs=d4)
sgd=SGD(lr=1e-8,decay=1e-9,momentum=0.9,nesterov=True)
aa=np.asarray([[0,1 ] if y_train[i] else [1,0] for i in range (len(y_train))])
aa=aa.reshape(len(y_train),2)

In [183]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=f1_m)

In [184]:
model.fit(X_train,aa,epochs=10,batch_size=4)

Epoch 1/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3701 - f1_m: 0.7881
Epoch 2/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3536 - f1_m: 0.7919
Epoch 3/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3651 - f1_m: 0.7914
Epoch 4/10
15763/15763 [==============================] - 19s 1ms/step - loss: 2.3582 - f1_m: 0.7912
Epoch 5/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3536 - f1_m: 0.7923
Epoch 6/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3512 - f1_m: 0.7920
Epoch 7/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3560 - f1_m: 0.7929
Epoch 8/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3521 - f1_m: 0.7922
Epoch 9/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3516 - f1_m: 0.7918
Epoch 10/10
15763/15763 [==============================] - 18s 1ms/step - loss: 2.3525 - f1

In [185]:
y_pred_NN=model.predict(X_test)
y_pred_NN=np.asarray([1 if y_pred_NN[i,1]>0.5 else 0 for i in range (y_pred_NN.shape[0])])

In [186]:
f1_score(y_test,y_pred_NN)

0.8008948545861297

(15763,)